# TCV 3151 – Computer Vision Lab (Practical Test)
## Public_dataset (metal, paper, plastic)
## Optimized for folder-based, transfer-friendly training

"**Classes:** metal, paper, plastic",

⚡ **Key Strategy:** Custom CNN architecture specifically designed for 32×32 CIFAR images
💡 **Why not ResNet50?** ResNet is designed for 224×224 ImageNet images and performs poorly on 32×32 images


## Section 1: Import Required Libraries

In [1]:
pip install numpy matplotlib tensorflow scikit-learn seaborn pillow h5py

Note: you may need to restart the kernel to use updated packages.


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.datasets import cifar100
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import (
    Conv2D, MaxPooling2D, Dense, Dropout, 
    Flatten, BatchNormalization, Activation
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns


print("✅ All libraries imported successfully!")
print(f"GPU Available: {len(tf.config.list_physical_devices('GPU')) > 0}")
print(f"TensorFlow Version: {tf.__version__}")

✅ All libraries imported successfully!
GPU Available: False
TensorFlow Version: 2.20.0


In [3]:
# Configure GPU (memory growth) and enable mixed precision when GPU is present
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✅ GPU devices found: {[g.name for g in gpus]}")
        # Enable mixed precision for faster training on modern GPUs
        try:
            from tensorflow.keras import mixed_precision
            mixed_precision.set_global_policy('mixed_float16')
            print("✅ Mixed precision enabled (mixed_float16)")
        except Exception as e:
            print("⚠️ Could not enable mixed precision:", e)
    except Exception as e:
        print("⚠️ Could not set memory growth for GPUs:", e)
else:
    print("⚠️ No GPU devices found. Training will run on CPU.")

⚠️ No GPU devices found. Training will run on CPU.


## Section 2: Load and Prepare CIFAR-100 Dataset (Classes 61-70)

In [ ]:
# Use Public_dataset folder with classes: metal, paper, plastic
DATA_DIR = 'Public_dataset'
IMG_SIZE = (512, 512)
BATCH_SIZE = 8

from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Training generator with a validation split
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    horizontal_flip=True,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    validation_split=0.1
)

train_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training',
    shuffle=True
)

val_generator = train_datagen.flow_from_directory(
    DATA_DIR,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation',
    shuffle=False
)

print(f"✅ Found classes: {train_generator.class_indices}")
print(f"Training samples: {train_generator.samples} | Validation samples: {val_generator.samples}")

Found 2417 images belonging to 3 classes.
Found 267 images belonging to 3 classes.
✅ Found classes: {'aluminium': 0, 'paper': 1, 'plastic': 2}
Training samples: 2417 | Validation samples: 267


In [4]:
# Define class names and count (required before later prints/evaluation)
CLASS_NAMES = list(train_generator.class_indices.keys())
NUM_CLASSES = train_generator.num_classes
print(f"Classes detected: {CLASS_NAMES} | Num classes: {NUM_CLASSES}")

Classes detected: ['aluminium', 'paper', 'plastic'] | Num classes: 3


**Note:** Dataset changed to `Public_dataset` (folders: `aluminium`, `paper`, `plastic`). Images are resized to **512×512** and rescaled by the generator. Model output updated to **3 classes**.

In [5]:
print('\n📊 Final Data Split:' )
print(f"Training: {train_generator.samples} samples")
print(f"Validation: {val_generator.samples} samples")
print('Test: (use separate test set if available)')
print(f"Classes: {CLASS_NAMES} | Num classes: {NUM_CLASSES}")


📊 Final Data Split:
Training: 2417 samples
Validation: 267 samples
Test: (use separate test set if available)
Classes: ['aluminium', 'paper', 'plastic'] | Num classes: 3


In [6]:
# Data augmentation handled by train_datagen (see earlier).
print("✅ Data augmentation configured via train_datagen (rescale + augmentations).")

✅ Data augmentation configured via train_datagen (rescale + augmentations).


## Section 4: Build Custom CNN (Adapted for Public_dataset images)

In [7]:
# Build Custom CNN Architecture (VGG-style)
model = Sequential([
    # Block 1
    Conv2D(64, (3, 3), padding='same', input_shape=(512, 512, 3)),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(64, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.2),

    # Block 2
    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(128, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.3),

    # Block 3
    Conv2D(256, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    Conv2D(256, (3, 3), padding='same'),
    BatchNormalization(),
    Activation('relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.4),

    # Classifier
    Flatten(),
    Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
    BatchNormalization(),
    Dropout(0.5),
    Dense(3, activation='softmax', dtype='float32')
])

print("✅ Custom CNN built successfully!")
print(f"   Total parameters: {model.count_params():,}")

c:\Users\USER\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


✅ Custom CNN built successfully!
   Total parameters: 104,504,643


In [ ]:
# Callbacks: EarlyStopping, ReduceLROnPlateau, ModelCheckpoint (saves best model)
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

early_stopping = EarlyStopping(monitor='val_accuracy', patience=10, restore_best_weights=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
model_checkpoint = ModelCheckpoint('best_model.h5', save_best_only=True, monitor='val_accuracy', mode='max')

callbacks = [early_stopping, reduce_lr, model_checkpoint]

print('✅ Callbacks configured: EarlyStopping, ReduceLROnPlateau, ModelCheckpoint')

In [ ]:
# Compile (if not compiled) and start training using directory generators
import tensorflow as tf
from tensorflow.keras.optimizers import Adam

EPOCHS = 50

if not hasattr(model, 'optimizer'):
    print('Compiling model...')
    model.compile(optimizer=Adam(learning_rate=1e-4), loss='categorical_crossentropy', metrics=['accuracy'])
else:
    print('Model already compiled.')

# Safety checks
if 'train_generator' not in globals():
    raise RuntimeError('train_generator not found. Make sure the data loading cell was executed and train_generator exists.')

steps_per_epoch = math.ceil(train_generator.samples / train_generator.batch_size)
validation_steps = math.ceil(val_generator.samples / val_generator.batch_size)

history = model.fit(
    train_generator,
    epochs=EPOCHS,
    steps_per_epoch=steps_per_epoch,
    validation_data=val_generator,
    validation_steps=validation_steps,
    callbacks=callbacks
)

print('✅ Training finished. Best weights should be saved to best_model.h5')

In [ ]:
# Inference on Dataset/test (or any split)
import os
import math
import pandas as pd
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import matplotlib.pyplot as plt

TEST_DIR = 'Dataset/test'
if not os.path.exists(TEST_DIR):
    raise FileNotFoundError(f"Test directory not found: {TEST_DIR}")

# Derive class names if missing
try:
    class_names = CLASS_NAMES
except NameError:
    # Try to infer from train_generator
    try:
        class_names = list(train_generator.class_indices.keys())
    except NameError:
        # Fallback: read subdirs
        class_names = sorted([d for d in os.listdir('Dataset') if os.path.isdir(os.path.join('Dataset', d))])

print(f"Classes used for inference: {class_names}")

# Create test generator
test_datagen = ImageDataGenerator(rescale=1./255)
# Use a small batch size for inference to avoid OOM
test_batch = min(8, globals().get('BATCH_SIZE', 8))

test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMG_SIZE,
    batch_size=test_batch,
    class_mode='categorical',
    shuffle=False
)

# Load best model if available
from tensorflow.keras.models import load_model
if os.path.exists('best_model.h5'):
    print('Loading best_model.h5')
    model_to_use = load_model('best_model.h5')
else:
    print('best_model.h5 not found, using in-memory model')
    model_to_use = model

steps = math.ceil(test_generator.samples / test_generator.batch_size)
preds = model_to_use.predict(test_generator, steps=steps, verbose=1)
pred_labels = preds.argmax(axis=1)
true_labels = test_generator.classes
filenames = test_generator.filenames

# Map indices to class names
idx_to_class = {v: k for k, v in test_generator.class_indices.items()}
pred_class_names = [idx_to_class[i] for i in pred_labels]
true_class_names = [idx_to_class[i] for i in true_labels]

# Save CSV
df = pd.DataFrame({'filename': filenames, 'true_label': true_class_names, 'pred_label': pred_class_names})
df.to_csv('predictions_test.csv', index=False)
print("✅ Predictions saved to predictions_test.csv")

# Classification report and confusion matrix
print('\nClassification Report:')
print(classification_report(true_labels, pred_labels, target_names=list(idx_to_class.values())))

cm = confusion_matrix(true_labels, pred_labels)
plt.figure(figsize=(6,6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=list(idx_to_class.values()), yticklabels=list(idx_to_class.values()))
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix (Test)')
plt.show()